In [1]:
# Parser
from os import listdir
from os.path import isfile, join

path_project_public_1 = './project_public_1/'

files_project_public_1 = [join(path_project_public_1, f) for f in listdir(path_project_public_1) if isfile(join(path_project_public_1, f))]

print(files_project_public_1)

['./project_public_1/bdbc_model.xml', './project_public_1/h264_interactions.txt', './project_public_1/h264_example_activation.txt', './project_public_1/bdbc_feature.txt', './project_public_1/h264.xml', './project_public_1/bdbc_interactions.txt', './project_public_1/h264_feature.txt']


In [57]:
class xml_parser:
    
    def linear_regression(self, n) :
        import itertools
        combinations = list(map(list, itertools.product([0, 1], repeat=n)))
        #print(combinations)
        return(combinations)
    
    def parse_file(self):
        bdbc_files = 'bdbc'
        h264 = 'h264'
        xml_files = '.xml'

        var_map = [] #

        final_string = "" #
        variables_string = "" #
        p_line = "p cnf " #
        clauses = "" #
        current_options = [] # 
        optional_false_elements = []
        clauses_2D = []

        for element in files_project_public_1:
            if bdbc_files in element and xml_files in element:
            #if h264 in element and xml_files in element:
                
                # open xml
                from lxml import etree
                xml_rules = etree.parse(open(element, 'r'))
                root = xml_rules.getroot()
                
                # first iteration to generate the dimacs variables
                for child in root :
                    for subchild in child :
                        for element in subchild : 
                            if element.tag == 'name':
                                var_map.append(element.text)
                
                # generate variables
                for el in var_map:
                    variables_string += "c " + str(1+var_map.index(el)) + " " + el + "\n"
                
                # generate rules
                for child in root :
                    for subchild in child :
                        for element in subchild :
                            
                            # temporary save current element and excluded options
                            if element.tag == 'name':
                                current_options.append(element.text)
                            if element.tag == 'excludedOptions' :
                                for option in element :
                                    current_options.append(option.text)
                            
                            # clear temp if 'otional' tag is set to true
                            if element.tag == 'optional' :
                                if element.text != 'False' :
                                    current_options = []
                        
                        # if there are choices to do generate all combinations at once
                        if len(current_options) > 0 :
                            ###########################################################################
                            # adding all combinations of rules if they are not already in 
                            ###########################################################################
                            # current_name, current_options
                            #
                            # print("Inside combinations loop", str(len(current_options)))

                            if len(current_options) == 1 :
                                #print("exact one element")
                                for element in current_options:
                                    #print("1")
                                    tmp=[]
                                    tmp.append(1+var_map.index(element))
                                    clauses_2D.append(tmp)
                            
                            if len(current_options) > 1 :
                                ####################################################current_options = current_options.sort()
                                combinations = self.linear_regression(len(current_options))
                                for lst in combinations :
                                    #print(str(len(current_options)))
                                    #print(len(combinations))
                                    tmp_list = []
                                    for el, el2 in zip(lst, current_options) :
                                        if el :
                                            tmp_list.append(1+var_map.index(el2))
                                        else :
                                            tmp_list.append((1+var_map.index(el2))*(-1))
                                    
                                    # do not put duplicate elements in list
                                    tmp_list.sort()
                                    is_in_list = False
                                    for item in clauses_2D:
                                        if item == tmp_list:
                                            is_in_list = True
                                    if not is_in_list :
                                        clauses_2D.append(tmp_list)
                                    
                        current_options = []

                p_line += str(len(var_map))+" "+str(len(clauses_2D))+"\n"
                
                #print(clauses_2D)
                for line in clauses_2D :
                    for item in line :
                        clauses += str(item)+" "
                    clauses += "0\n"

                # und zack fertig .... dimacs
                final_string = variables_string+p_line+clauses
                return final_string

class population_tool:
    
    
    def init_population(self, population, constraints) :
        import random
        
        pop = population
        configuration = []
        tabu_config = []
        
        is_valid = False
        
        # init size, value of arrays
        for el in pop:
            configuration.append(False)
            tabu_config.append(False)
        
        # do random initial solution
        for idx in range(len(configuration)) :
            configuration[idx] = bool(random.getrandbits(1))
        
        # check validity of solution here and add valid parts to tabu config
        if (random.random() > 0.9) :
            is_valid = True
        #is_valid = check validity
        
        # simulate valid parts
        for idx in range(len(tabu_config)) :
            if (tabu_config[idx] == False and random.random() > 0.4) :
                tabu_config[idx]=True
        
        while not is_valid :
            for idx in range(len(configuration)) :
                if not tabu_config[idx]:
                    configuration[idx] = bool(random.getrandbits(1))
            
            if (random.random() > 0.95 or len(tabu_config) == tabu_config.count(True)) :
                is_valid = True
                
            # simulate valid parts
            for idx in range(len(tabu_config)) :
                if (tabu_config[idx] == False and random.random() > 0.4) :
                    tabu_config[idx]=True

            # check validity of solution here and add valid parts to tabu config
            #is_valid = check validity
        return configuration
            
parser = xml_parser()
#print(parser.parse_file())
#print("\n")

pop_obj = population_tool()
pop = [1,2,3,4,5]
constr = [[-1],[2, -3],[4,5]]
print(pop_obj.init_population(pop, constr))




[False, True, True, False, False]
